<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#試算表個人記帳(作業一)
##使用者輸入
- 可自行加入日期時間或按enter輸入現在時間。
- 輸入品項後可選擇屬於哪個種類，與輸入金額。
##統計消費總額
- 可計算出當日總額和當月總額。
- 製作各種類總花費圓餅圖。
- [試算表連結](https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=0#gid=0)

In [27]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import calendar


In [28]:
# === 驗證 Google Sheets ===
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=0#gid=0')

worksheet = gsheets.worksheet("工作表1")

# === 檢查標題列 ===
headers = ["日期", "時間", "品項", "種類", "金額"]
first_row = worksheet.row_values(1)
if not first_row or first_row != headers:
    worksheet.clear()
    worksheet.append_row(headers)


In [34]:
# === 使用者輸入 ===
date_str = input("請輸入日期 (格式：YYYY-MM-DD, Enter=今天): ").strip()
if date_str == "":
    date_str = datetime.now().strftime("%Y-%m-%d")
else:
    datetime.strptime(date_str, "%Y-%m-%d")

time_str = input("請輸入時間 (格式：HH:MM, Enter=現在時間): ").strip()
if time_str == "":
    time_str = datetime.now().strftime("%H:%M")
else:
    datetime.strptime(time_str, "%H:%M")

item = input("請輸入品項: ").strip()
amount = float(input("請輸入金額: "))

categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]
print("請選擇種類：")
for i, c in enumerate(categories, 1):
    print(f"{i}. {c}")
choice = int(input("輸入數字選擇: "))
category = categories[choice - 1]

# === 新增資料到 Google Sheets (RAW 保證金額正確) ===
worksheet.append_row([date_str, time_str, item, category, amount], value_input_option="RAW")
print("✅ 新增成功！")


請輸入日期 (格式：YYYY-MM-DD, Enter=今天): 
請輸入時間 (格式：HH:MM, Enter=現在時間): 
請輸入品項: 咖哩
請輸入金額: 120
請選擇種類：
1. 早餐
2. 午餐
3. 晚餐
4. 點心
5. 飲料
6. 衣服
7. 娛樂
8. 交通
輸入數字選擇: 3
✅ 新增成功！


In [35]:
# === 讀取 DataFrame ===
sheets = worksheet.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")

In [36]:
# === 更新每日總支出 ===
def update_day_total(df, worksheet, date_str):
    target_data = df[(df["日期_str"] == date_str) & (df["品項"] != "當天總支出")]
    day_total = target_data["金額"].sum()

    # 找出當日最後一筆 row
    rows = worksheet.findall(date_str)
    last_row_day = rows[-1].row if rows else len(worksheet.get_all_values())

    sheet_rows = worksheet.get_all_values()

    # 更新或新增「當天總支出」
    if last_row_day < len(sheet_rows) and "當天總支出" in sheet_rows[last_row_day]:
        cell = f"E{last_row_day + 1}"  # 第 5 欄
        worksheet.update(cell, [[day_total]], value_input_option="RAW")
        target_row = last_row_day + 1
    else:
        worksheet.insert_row([date_str, "", "當天總支出", "", day_total],
                             index=last_row_day + 1,
                             value_input_option="RAW")
        target_row = last_row_day + 1

    print(f"📌 {date_str} 的總支出已更新：{int(day_total)} 元")
    return day_total
# 更新每日總結
day_total = update_day_total(df, worksheet, date_str)


📌 2025-09-17 的總支出已更新：185 元


In [16]:
# === 更新每月總支出 ===
def update_month_total(df, date_str):
    year = int(date_str[:4])
    month = int(date_str[5:7])
    month_total = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]["金額"].sum()
    print(f"📌 {date_str[:7]} 當月總支出：{int(month_total)} 元")
    return year, month
# 更新每月總結
year, month = update_month_total(df, date_str)

📌 2025-09 當月總支出：890 元


In [37]:
# === 更新圓餅圖 & 類別統計表 ===
def update_pie_chart(df, worksheet, year, month):
    # 篩選當月資料
    month_data = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]

    # 確保欄位唯一
    month_data = month_data.loc[:, ~month_data.columns.duplicated()]

    # 檢查必要欄位
    if "種類" not in month_data.columns or "金額" not in month_data.columns:
        print("⚠️ 缺少『種類』或『金額』欄位，無法更新統計")
        return

    # 金額轉數字
    month_data["金額"] = pd.to_numeric(month_data["金額"], errors="coerce").fillna(0)

    # 按種類統計總金額
    cat_summary = month_data.groupby("種類", as_index=False)["金額"].sum()

    # === 寫入統計表到 G 欄 ===
    worksheet.update("G1", [["種類", "總金額"]])  # 標題
    if not cat_summary.empty:
        worksheet.update(f"G2:H{len(cat_summary)+1}", cat_summary.values.tolist())

    # === 建立圓餅圖 (會覆蓋舊的) ===
    chart_request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": f"{year}-{month:02d} 各種類支出分布",
                    "pieChart": {
                        "legendPosition": "RIGHT_LEGEND",
                        "domain": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 0,
                                    "endRowIndex": len(cat_summary) + 1,
                                    "startColumnIndex": 6,  # G 欄
                                    "endColumnIndex": 7
                                }]
                            }
                        },
                        "series": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 0,
                                    "endRowIndex": len(cat_summary) + 1,
                                    "startColumnIndex": 7,  # H 欄
                                    "endColumnIndex": 8
                                }]
                            }
                        },
                        "pieHole": 0.3  # 甜甜圈
                    }
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {
                            "sheetId": worksheet.id,
                            "rowIndex": 0,
                            "columnIndex": 9  # 圖表放在 J 欄
                        }
                    }
                }
            }
        }
    }

    gsheets.batch_update({"requests": [chart_request]})
    print("✅ 種類統計表 & 圓餅圖已更新！")
# ✅ 更新種類統計表 & 圓餅圖
update_pie_chart(df, worksheet, year, month)

/tmp/ipython-input-2513705454.py:21: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("G1", [["種類", "總金額"]])  # 標題
/tmp/ipython-input-2513705454.py:23: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(f"G2:H{len(cat_summary)+1}", cat_summary.values.tolist())


✅ 種類統計表 & 圓餅圖已更新！
